In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
import numpy as np
import pickle 
import pandas as pd


In [20]:
final = pd.read_csv(r'final1.csv')

In [21]:
final.shape

(145901, 8)

In [22]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145901 entries, 0 to 145900
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     145901 non-null  int64  
 1   sender         145901 non-null  object 
 2   receiver       145901 non-null  object 
 3   date           145901 non-null  object 
 4   subject_clean  144817 non-null  object 
 5   body_clean     145106 non-null  object 
 6   label          145901 non-null  int64  
 7   urls           95901 non-null   float64
dtypes: float64(1), int64(2), object(5)
memory usage: 8.9+ MB


In [23]:
final['text'] = final['subject_clean'] + " " + final['body_clean']


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
import numpy as np


texts = final['text'].astype(str).values
labels = final['label'].values

X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)


tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


max_len = 300

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')


model = Sequential()
model.add(Embedding(50000, 128, input_length=max_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


history = model.fit(
    X_train_pad,
    y_train,
    validation_data=(X_test_pad, y_test),
    epochs=5,
    batch_size=64
)


loss, acc = model.evaluate(X_test_pad, y_test)
print("Accuracy:", round(acc*100,2), "%")


c:\Users\Harsh\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1824/1824 ━━━━━━━━━━━━━━━━━━━━ 1460s 795ms/step - accuracy: 0.7232 - loss: 0.5439 - val_accuracy: 0.9728 - val_loss: 0.0917
Epoch 2/5
1824/1824 ━━━━━━━━━━━━━━━━━━━━ 2333s 1s/step - accuracy: 0.9791 - loss: 0.0656 - val_accuracy: 0.9799 - val_loss: 0.0525
Epoch 3/5
1824/1824 ━━━━━━━━━━━━━━━━━━━━ 1607s 881ms/step - accuracy: 0.9882 - loss: 0.0317 - val_accuracy: 0.9837 - val_loss: 0.0492
Epoch 4/5
1824/1824 ━━━━━━━━━━━━━━━━━━━━ 3784s 2s/step - accuracy: 0.9925 - loss: 0.0204 - val_accuracy: 0.9810 - val_loss: 0.0490
Epoch 5/5
1824/1824 ━━━━━━━━━━━━━━━━━━━━ 1553s 852ms/step - accuracy: 0.9943 - loss: 0.0169 - val_accuracy: 0.9786 - val_loss: 0.0545
912/912 ━━━━━━━━━━━━━━━━━━━━ 117s 128ms/step - accuracy: 0.9799 - loss: 0.0525
Accuracy: 97.86 %


In [26]:
model.save("phishing_lstm_model.h5")

In [28]:
import pickle
pickle.dump(tokenizer, open("tokenizer.pkl", "wb"))


In [29]:
print(final['label'].value_counts(dropna=False))
print(final['label'].unique())


label
0    94243
1    51658
Name: count, dtype: int64
[0 1]
